**TODO :**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src

/home/theo/kaggle/foursquare/src


In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# **Librairies**

In [4]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch

torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

<IPython.core.display.Javascript object>

In [5]:
import gc
import cudf
import random
import warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pandarallel import pandarallel

from params import DEBUG, OUT_PATH, IS_TEST, RESSOURCES_PATH
from ressources import *
from matching import *
from dtypes import DTYPES_1
from data.preparation import reduce_mem_usage

pandarallel.initialize(nb_workers=12, progress_bar=False)
random.seed(13)
warnings.simplefilter("ignore")
pd.options.display.max_columns = 500

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


<IPython.core.display.Javascript object>

## Load Data
- TODO: load precomputed probas

In [6]:
if IS_TEST:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_test.csv")
else:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_train.csv")

<IPython.core.display.Javascript object>

In [7]:
train["idx"] = np.arange(len(train))

<IPython.core.display.Javascript object>

In [8]:
THRESHOLD = 0.0075

<IPython.core.display.Javascript object>

In [9]:
if IS_TEST:
    df_p = pd.read_csv(
        OUT_PATH + f"features_test_1_filtered_{THRESHOLD}.csv", dtype=DTYPES_1
    )
    SIZE_RATIO = 1
else:
    df_p = pd.read_csv(
        OUT_PATH + f"features_train_1_filtered_{THRESHOLD}.csv", dtype=DTYPES_1
    )
    SIZE_RATIO = 1

<IPython.core.display.Javascript object>

In [10]:
if DEBUG:
    df_p = df_p.head(10000).copy()

<IPython.core.display.Javascript object>

In [11]:
# df_merged = feature_engineering_2(df_p, train, ressources_path=RESSOURCES_PATH)

<IPython.core.display.Javascript object>

### Youri & Vincent

In [12]:
from fe import FE2

<IPython.core.display.Javascript object>

In [13]:
df_p = df_p.merge(train[["id", "Nb_multiPoi"]], left_on="id_1", right_on="id").drop(
    "id", axis=1
)
df_p = df_p.merge(
    train[["id", "Nb_multiPoi"]], left_on="id_2", right_on="id", suffixes=("_1", "_2")
).drop("id", axis=1)

<IPython.core.display.Javascript object>

In [14]:
p1 = df_p[["id_1"]].copy()
p1.columns = ["id"]
p2 = df_p[["id_2"]].copy()
p2.columns = ["id"]

<IPython.core.display.Javascript object>

In [15]:
%%time
df = FE2(df_p.copy(), p1, p2, train, RESSOURCES_PATH, size_ratio=SIZE_RATIO)

- Cat links & quantiles
- Simply connected components
- Strongly connected components
- Cat link score
- Link between grouped categories
- Features for column name_initial
- Features for column name_initial_decode
- Features for column nameC
- Features for column name
- Features for column categories
- Features for column address
- Features for column url
- Features for column city
- Features for column state
- Features for column zip
- Features for column phone
- Count encoding
- Words in categories
- Words in names
- Close count
- Close count of same category2
- Close count of same category_simpl
- Compare numeric part of the name/address
- Number in names features
CPU times: user 26min 58s, sys: 1min 59s, total: 28min 58s
Wall time: 34min 14s


<IPython.core.display.Javascript object>

### Théo

In [16]:
from fe_theo import feature_engineering_theo_2

<IPython.core.display.Javascript object>

In [17]:
cols = [
    "id",
    "name",
    "address",
    "country",
    "url",
    "phone",
    "city",
    "state",
    "zip",
    "idx",
]

for col in cols[1:]:
    train.loc[train[col] == "", col] = np.nan

p1 = p1[["id"]].merge(train[cols], on="id", how="left")
p2 = p2[["id"]].merge(train[cols], on="id", how="left")

pairs = pd.concat([p1[cols], p2[cols]], axis=1)
pairs.columns = [c + "_1" for c in cols] + [c + "_2" for c in cols]

<IPython.core.display.Javascript object>

In [18]:
%%time

df_theo, fts_theo = feature_engineering_theo_2(train.copy(), pairs.copy(), cuda=False)

- Computing nan features
- Computing feature name_tf_idf_33_char_wb_sim
- Computing feature address_tf_idf_33_char_wb_sim
- Computing feature url_tf_idf_33_char_wb_sim
- Column : name  -  Function : wratio
- Column : name  -  Function : partial_ratio
- Column : address  -  Function : wratio
- Column : address  -  Function : partial_ratio
- Column : url  -  Function : wratio
- Column : url  -  Function : partial_ratio
CPU times: user 1min, sys: 16.2 s, total: 1min 16s
Wall time: 1min 54s


<IPython.core.display.Javascript object>

## Merge & Save

In [19]:
df_merged = df.merge(df_theo, on=["id_1", "id_2"])

<IPython.core.display.Javascript object>

In [20]:
cols_to_end = [
    "point_of_interest_1",
    "fold_1",
    "point_of_interest_2",
    "fold_2",
    "match",
]
cols_to_end = [c for c in cols_to_end if c in df_merged.columns]

if len(cols_to_end):
    to_end = df_merged[cols_to_end]
    df_merged.drop(cols_to_end, axis=1, inplace=True)
    df_merged[cols_to_end] = to_end

<IPython.core.display.Javascript object>

In [21]:
if DEBUG:
    corrs = df_merged.corr()
    for col in corrs.columns:
        close = pd.DataFrame(corrs.loc[corrs[col] > 0.99][col])
        if len(close) > 1:
            display(close[close[col] < 1])

<IPython.core.display.Javascript object>

In [22]:
df_merged = reduce_mem_usage(df_merged)

Memory usage after optimization is: 1888.75 MB
Decreased by 47.9%


<IPython.core.display.Javascript object>

In [23]:
# df_merged.dtypes.to_dict()

<IPython.core.display.Javascript object>

In [ ]:
if not DEBUG:
    if IS_TEST:
        df_merged.to_csv(OUT_PATH + f"features_test_2_{THRESHOLD}.csv", index=False)
    else:
        df_merged.to_csv(OUT_PATH + f"features_train_2_{THRESHOLD}.csv", index=False)

In [ ]:
df_merged.head()

Done !